In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_io as tfio
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

from skimage import exposure, io, img_as_ubyte
from skimage.morphology import disk
from skimage.filters import rank
from skimage.restoration import denoise_bilateral

import skimage.exposure

import tf_clahe
import os
import cv2
print(tf.__version__)

In [ ]:
# Homomorphic filter class
class HomomorphicFilter:
    """Homomorphic filter implemented with diferents filters and an option to an external filter.
    
    High-frequency filters implemented:
        butterworth
        gaussian
    Attributes:
        a, b: Floats used on emphasis filter:
            H = a + b*H
        
        .
    """

    def __init__(self, a = 0.5, b = 1.5):
        self.a = float(a)
        self.b = float(b)

    # Filters
    def __butterworth_filter(self, I_shape, filter_params):
        P = I_shape[0]/2
        Q = I_shape[1]/2
        U, V = np.meshgrid(range(I_shape[0]), range(I_shape[1]), sparse=False, indexing='ij')
        Duv = (((U-P)**2+(V-Q)**2)).astype(float)
        H = 1/(1+(Duv/filter_params[0]**2)**filter_params[1])
        return (1 - H)

    def __gaussian_filter(self, I_shape, filter_params):
        P = I_shape[0]/2
        Q = I_shape[1]/2
        H = np.zeros(I_shape)
        U, V = np.meshgrid(range(I_shape[0]), range(I_shape[1]), sparse=False, indexing='ij')
        Duv = (((U-P)**2+(V-Q)**2)).astype(float)
        H = np.exp((-Duv/(2*(filter_params[0])**2)))
        return (1 - H)

    # Methods
    def __apply_filter(self, I, H):
        H = np.fft.fftshift(H)
        I_filtered = (self.a + self.b*H)*I
        return I_filtered

    def filter(self, I, filter_params, filter='butterworth', H = None):
        """
        Method to apply homormophic filter on an image
        Attributes:
            I: Single channel image
            filter_params: Parameters to be used on filters:
                butterworth:
                    filter_params[0]: Cutoff frequency 
                    filter_params[1]: Order of filter
                gaussian:
                    filter_params[0]: Cutoff frequency
            filter: Choose of the filter, options:
                butterworth
                gaussian
                external
            H: Used to pass external filter
        """

        #  Validating image
        if len(I.shape) != 2:
            raise Exception('Improper image')

        # Take the image to log domain and then to frequency domain 
        I_log = np.log1p(np.array(I, dtype="float"))
        I_fft = np.fft.fft2(I_log)

        # Filters
        if filter=='butterworth':
            H = self.__butterworth_filter(I_shape = I_fft.shape, filter_params = filter_params)
        elif filter=='gaussian':
            H = self.__gaussian_filter(I_shape = I_fft.shape, filter_params = filter_params)
        elif filter=='external':
            print('external')
            if len(H.shape) != 2:
                raise Exception('Invalid external filter')
        else:
            raise Exception('Selected filter not implemented')
        
        # Apply filter on frequency domain then take the image back to spatial domain
        I_fft_filt = self.__apply_filter(I = I_fft, H = H)
        I_filt = np.fft.ifft2(I_fft_filt)
        I = np.exp(np.real(I_filt))-1
        return np.float32(I)

In [ ]:
def depth_estimation(inputFile, outputFile):
    # load image as grayscale
    # print(inputFile)
    img = cv2.imread(inputFile, cv2.IMREAD_ANYDEPTH)

    # stretch to full dynamic range
    stretch = skimage.exposure.rescale_intensity(img, in_range='image', out_range=(0,255)).astype(np.uint8)

    # convert to 3 channels
    stretch = cv2.merge([stretch,stretch,stretch])

    cv2.imwrite(outputFile, stretch)
    
def contrast_depth_estimation(inputFile, outputFile):
    # load image as grayscale
    img = cv2.imread(inputFile, cv2.IMREAD_ANYDEPTH)
    
    # original = img.copy()
    xp = [0, 64, 128, 192, 255]
    fp = [0, 16, 128, 240, 255]
    x = np.arange(256)
    table = np.interp(x, xp, fp).astype('uint8')
    img = cv2.LUT(img, table)
    
    
    stretch = cv2.merge([img,img,img])
    
    cv2.imwrite(outputFile, stretch)


def convert_contrast(inputFile, outputFile):

    img = tf.io.read_file(inputFile)
    img = tf.io.decode_png(img, channels=1)
    # print(tf.rank(img))
    img = tf.cast(img, tf.float32)

    
    img =  tf.image.adjust_contrast(img, 0.25)

    tf.keras.utils.save_img(outputFile, img)

def convert_file_clahe(inputFile, outputFile):
    bgr = cv2.imread(inputFile)

    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)

    lab_planes = cv2.split(lab)

    clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(8,9))
    # img = clahe.apply(lab)
    eq_channels = []
    for ch in lab_planes:
        eq_channels.append(clahe.apply(ch))

    eq_image = cv2.merge(eq_channels)

    bgr = cv2.cvtColor(eq_image, cv2.COLOR_LAB2BGR)
    cv2.imwrite(outputFile, bgr)

def convert_grayscale(inputFile, outputFile):
    bgr = cv2.imread(inputFile)

    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)

    cv2.imwrite(outputFile, lab)
    
def convert_file_global_equal(inputFile, outputFile):
    img = cv2.imread(inputFile)
    # img = skimage.io.imread(fname=inputFile)
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

    lab_planes = cv2.split(img)
    eq_channels = []
    for ch in lab_planes:
        eq_channels.append(exposure.equalize_hist(ch))
    
    eq_image = cv2.merge(eq_channels)
    # bgr = cv2.cvtColor(eq_image, cv2.COLOR_LAB2BGR)
    
    io.imsave(fname=outputFile, arr=img_as_ubyte(eq_image))

def convert_file_local_equal(inputFile, outputFile):
    img = cv2.imread(inputFile)
    # img = skimage.io.imread(fname=inputFile)
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    
    lab_planes = cv2.split(img)
    eq_channels = []
    selem = disk(30)
    for ch in lab_planes:
        eq_channels.append(rank.equalize(ch, selem=selem))
    
    eq_image = cv2.merge(eq_channels)
    bgr = cv2.cvtColor(eq_image, cv2.COLOR_LAB2BGR)
    
    io.imsave(fname=outputFile, arr=img_as_ubyte(img))
    
def convert_denoise(inputFile, outputFile):
    img = cv2.imread(inputFile)
    
    img = denoise_bilateral(img_as_ubyte(img), sigma_color=0.1, sigma_spatial=15, multichannel=True)
    
    io.imsave(fname=outputFile, arr=img_as_ubyte(img))

def convert_custom_method(inputFile, outputFile):
    img = tf.io.read_file(inputFile)
    img = tf.io.decode_bmp(img, channels=3)
    # print(tf.rank(img))
    img = tf.cast(img, tf.float32)

    img = tfio.experimental.color.rgb_to_bgr(img)
    img = tf.image.adjust_contrast(img, 11.)
    img = tf.image.adjust_hue(img, 11.)
    img = tf.image.adjust_gamma(img)
    img = tfa.image.median_filter2d(img)
    
    tf.keras.utils.save_img(outputFile, img)

def convert_image_format(inputFile, ext, dest_ext):
    
    if inputFile.endswith(ext): 
        prefix = inputFile.split(ext)[0]
        im = Image.open(inputFile)
        # print(prefix + dest_ext)
        im.save(prefix + dest_ext) 


kernel_size = (20, 20)
def histo_equalize(img):
    lab_planes = cv2.split(img)
    eq_channels = []
    for ch in lab_planes:
        eq_channels.append(cv2.equalizeHist(ch))

    img = cv2.merge(eq_channels)

    img = cv2.cvtColor(img, cv2.COLOR_LAB2BGR)
    return img

def opening_ops(img):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    return img

def closing_ops(img):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)
    img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    return img

def gabor_filter(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    g_kernel = cv2.getGaborKernel(kernel_size, 8.0, np.pi/4, 10.0, 0.5, 0, ktype=cv2.CV_32F)
    img = cv2.filter2D(img, cv2.CV_8UC3, g_kernel)
    return img

def convert_custom_method_v2(inputFile, outputFile):
    img = cv2.imread(inputFile)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

    img = opening_ops(img)
    img = closing_ops(img)
    img = histo_equalize(img)
    img = gabor_filter(img)
    
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    cv2.imwrite(outputFile, img)

def convert_sobelx(inputFile, outputFile):
    img = cv2.imread(inputFile)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    img = cv2.Sobel(img, cv2.CV_64F,1,0,ksize=5)
    # img = cv2.cvtColor(img, cv2.COLOR_LAB2BGR)
    cv2.imwrite(outputFile, img)
    # return img
    
def resize_image(inputFile, outputFile):
    # print(inputFile)
    img = cv2.imread(inputFile)
    
    img = cv2.resize(img, (481, 271))
    
    cv2.imwrite(outputFile, img)
    
def convert_sobelxy(inputFile, outputFile):
    img = cv2.imread(inputFile)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.Sobel(img,cv2.CV_32F,1,1,ksize=3)
    # img = cv2.Sobel(img,cv2.CV_32F,0,1,ksize=3)
    cv2.imwrite(outputFile, img)
    
def convert_custom_v3(inputFile, outputFile):
    img = cv2.imread(inputFile)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # img = cv2.GaussianBlur(img,(3,3),0)
    img = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=3)
    img = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=3)
    # img = cv2.cvtColor(img, cv2.COLOR_LAB2BGR)
    cv2.imwrite(outputFile, img)
    # return img


def homo_filter(inputFile, outputFile):
    # Main code
    img = cv2.imread(inputFile)[:, :, 0]
    homo_filter = HomomorphicFilter(a = 0.5, b = 1.5)
    img_filtered = homo_filter.filter(I=img, filter_params=[3,3])
    
    cv2.imwrite(outputFile, img_filtered)

In [ ]:
## convert colour of images
for mode in ["test_data","train_data"]:
    for class_name in ["normal", "defect"]:
        Input_dir = f'mura_data/RGB/mura_clean/{mode}/{class_name}/'
        Out_dir = f'mura_data/RGB/mura_homo_filter/{mode}/{class_name}/'
        a = os.listdir(Input_dir)
        index = 0
        for i in a:
            index += 1
            if i != ".DS_Store" and i != ".ipynb_checkpoints":

                inputFile = Input_dir+i
                OutputFile = Out_dir+i

                # print(inputFile)
                # convert_file_bcet(inputFile, OutputFile)
                # convert_file_clahe(inputFile, OutputFile)
                # resize_image(inputFile, OutputFile)
                # convert_sobelx(inputFile, OutputFile)
                # convert_sobelxy(inputFile, OutputFile)
                # depth_estimation(inputFile, OutputFile)
                # contrast_depth_estimation(inputFile, OutputFile)
                homo_filter(inputFile, OutputFile)
                # convert_custom_method_v2(inputFile, OutputFile)
                # convert_file_clahe_v2(inputFile, OutputFile)
                # convert_image_format(inputFile, ".bmp", ".png")
                # convert_custom_method(inputFile, OutputFile)
                # convert_file_global_equal(inputFile, OutputFile)
                # convert_file_local_equal(inputFile, OutputFile)
                # convert_grayscale(inputFile, OutputFile)
                # convert_contrast(inputFile, OutputFile)
                # convert_denoise(inputFile, OutputFile)
                # convert_custom_v3(inputFile, OutputFile)
            if index % 1000 == 0:
                print("file: ",index)
        print("done.", class_name, mode)
    #     break

In [ ]:
print("test")